In [1]:
import pandas as pd
import torch
from torch.autograd import Variable
import os
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:

tracknames = os.listdir("/home/jasper-ubuntu/Documents/Studie/Master AI/Computational intelligence/Torcs-CI/CI/train_data/")
# tracknames = ['aalborg.csv']
datadictin = {}
datadictout = {}
framesin =[]
framesout=[]
for i, file in  enumerate(tracknames):
    path = os.path.join('train_data', file)
    print(path)
    datadictin[i] = pd.read_csv(path,sep=';', index_col=False).iloc[:-1, 3:]
    datadictout[i] = pd.read_csv(path,sep=';', index_col=False).iloc[:-1, 0:3]


#Concatenate all datasets
for i,file in enumerate(tracknames):
    framesin.append(datadictin[i])
    framesout.append(datadictout[i])
    
inp = pd.concat(framesin) 
outp = pd.concat(framesout) 

outp.head()

#Drop missing values
inp.replace('', np.nan, inplace=True)
inp.dropna(inplace=True)
outp.replace('', np.nan, inplace=True)
outp.dropna(inplace=True)
outp.head()

train_data/road_alpine-1_7cars_2475231.csv
train_data/road_alpine-1_7cars_2475119.csv
train_data/road_alpine-1_7cars_2475043.csv
train_data/road_alpine-1_7cars_2475155.csv


,accel,brake,steer
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0


In [39]:
# # Make 1 axis of brake and accelaration
# outp.loc[:,'BRAKE'] *= -1
# outp.iloc[:5,:2]

In [40]:
# maxCol= lambda x: max(x.min(), x.max(), key=abs)
# outp.iloc[:,0:2].apply(maxCol,axis=1)

# outp['brake_acceleration'] = pd.Series(max(abs(outp.iloc[:,0:2])))
outp.head()

,ACCELERATION,BRAKE,STEERING
0,1,0.0,-0.002715
1,1,0.0,-0.002715
2,1,0.0,-0.002715
3,1,0.0,-0.002715
4,1,0.0,-0.002715


In [3]:
colsin = inp.shape[1]
colsout = outp.shape[1]
datapoints = inp.shape[0]
print(datapoints,colsin,colsout)

132708 72 3


In [4]:
# Get data from pandas in a Tensor Variable
x = Variable(torch.from_numpy(inp.iloc[:,:].as_matrix()).float())
y = Variable(torch.from_numpy(outp.iloc[:,:].as_matrix()).float())
print(x)
print(y)

Variable containing:
     0.0000     -0.9820   6306.6499  ...     200.0000    200.0000    200.0000
     0.0000     -0.9620   6306.6499  ...     200.0000    200.0000    200.0000
     0.0000     -0.9420   6306.6499  ...     200.0000    200.0000    200.0000
                ...                   ⋱                   ...                
    -0.0359     90.9260   2000.6600  ...     200.0000    200.0000    200.0000
    -0.0366     90.9480   2000.9600  ...     200.0000    200.0000    200.0000
    -0.0363     90.9700   2001.2700  ...     200.0000    200.0000    200.0000
[torch.FloatTensor of size 132708x72]

Variable containing:
 1.0000  0.0000  0.0000
 1.0000  0.0000  0.0000
 1.0000  0.0000  0.0000
           ⋮            
 1.0000  0.0000 -0.0226
 1.0000  0.0000 -0.0759
 1.0000  0.0000 -0.0557
[torch.FloatTensor of size 132708x3]



In [20]:
# class Net(nn.Module):
#     def __init__(self, input_size, hidden_size, num_classes):
#         super(Net, self).__init__()
#         self.fc1 = nn.Linear(input_size, hidden_size) 
#         self.relu = nn.ReLU()
#         self.fc2 = nn.Linear(hidden_size, num_classes)  
    
#     def forward(self, x):
#         out = self.fc1(x)
#         out = self.relu(out)
#         out = self.fc2(out)
# return out

class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        
        h_relu = self.linear1(x).clamp(min=0)
        y = self.linear2(h_relu)
        y_pred = F.tanh(y)
        return y_pred

In [39]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 50, colsin, 500, colsout

# Use the nn package to define our model as a sequence of layers. nn.Sequential
# is a Module which contains other Modules, and applies them in sequence to
# produce its output. Each Linear Module computes output from input using a
# linear function, and holds internal Variables for its weight and bias.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
    torch.nn.Tanh()
)

# model = TwoLayerNet(D_in, H, D_out)


# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.

learning_rate = 1e-4

loss_fn = torch.nn.MSELoss(size_average=True)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
print(model.parameters())

for t in range(500):
    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Variable of input data to the Module and it produces
    # a Variable of output data.
    y_pred = model(x)

    # Compute and print loss. We pass Variables containing the predicted and true
    # values of y, and the loss function returns a Variable containing the
    # loss.
    loss = loss_fn(y_pred, y)
    print(t, loss.data[0])

    # Zero the gradients before running the backward pass.
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Variables with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()
    

    # Update the weights using gradient descent. Each parameter is a Variable, so
    # we can access its data and gradients like we did before.
    optimizer.step()


epoch : 0
learning rate : 1.0
epoch : 1
learning rate : 0.6
epoch : 2
learning rate : 0.36
epoch : 3
learning rate : 0.21599999999999997
epoch : 4
learning rate : 0.1296
epoch : 5
learning rate : 0.07775999999999998
epoch : 6
learning rate : 0.04665599999999999
epoch : 7
learning rate : 0.027993599999999993
epoch : 8
learning rate : 0.016796159999999994
epoch : 9
learning rate : 0.010077695999999997
epoch : 10
learning rate : 0.006046617599999997
epoch : 11
learning rate : 0.0036279705599999985
epoch : 12
learning rate : 0.002176782335999999
epoch : 13
learning rate : 0.0013060694015999993
epoch : 14
learning rate : 0.0007836416409599996


In [6]:
save_path = 'trainedmodel_4timesalpine1_500hidden_3out(tanh)200it_standardmodel_savedasmodel'
# https://stackoverflow.com/questions/42703500/best-way-to-save-a-trained-model-in-pytorch
torch.save(model, save_path)
# model.save_state_dict('trainedmodel_bbdata_500hidden_3out(tanh)_22in_500it_adam(hopefully_good_results.pt')

In [27]:
neural_net = torch.load('trainedmodel_bbdata_500hidden_3out(tanh)_21in_1000it')

In [40]:
value = x[100]

print(value)
print(model(value))
print(y[100])
# print(loss_fn(model(value),y[100]))

Variable containing:
   -0.0101
    1.0440
 6313.7100
    7.0576
    1.0000
    0.0000
    7.0000
 8935.3701
   50.1658
    0.0429
   -0.0253
    0.3401
    0.2795
   42.2622
   41.8815
   63.4362
   63.4362
    3.9594
    4.9131
    8.2495
   14.0993
   22.9128
   35.5212
   52.9234
   76.0210
  105.2300
  200.0000
  200.0000
  200.0000
  147.3400
   88.0843
   52.6279
   30.7736
   17.4026
   10.1312
    8.0412
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
   29.8444
   13.4412
   21.8065
    6.9078
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
  200.0000
[torch.FloatTensor of size 72]

Variable containing:
 0.1075
 0.2318
 0.2535
[torch.FloatTensor of size 3]

Variable containing:
 1.0000
 0.0000
 0.0196
[torch.FloatTensor of size 3]

